In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Atlas

In [37]:
Atlas = pd.read_excel('Atlas 2013_municipal, estadual e Brasil.xlsx', sheet_name='UF 91-00-10')[[ 'UFN', 'ANO', 'MORT5', 'POP']]
Atlas.columns = ['Local', 'Ano', 'Morte5', 'Populacao']
Atlas['Nível'] = 'UF'
Atlas.head(10)

,Local,Ano,Morte5,Populacao,Nível
0,Rondônia,1991,54.06,1120423,UF
1,Acre,1991,50.63,412309,UF
2,Amazonas,1991,61.49,2058501,UF
3,Roraima,1991,63.11,191382,UF
4,Pará,1991,64.44,4863829,UF
5,Amapá,1991,54.31,284283,UF
6,Tocantins,1991,83.26,904011,UF
7,Maranhão,1991,106.43,4891092,UF
8,Piauí,1991,84.80,2551985,UF
9,Ceará,1991,82.70,6286483,UF


In [38]:
regioes = {
    'Norte': ['Rondônia', 'Acre', 'Amazonas', 'Roraima', 'Pará', 'Amapá', 'Tocantins'],
    'Nordeste': ['Maranhão', 'Piauí', 'Ceará', 'Rio Grande do Norte', 'Paraíba', 'Pernambuco', 'Alagoas', 'Sergipe', 'Bahia'],
    'Sudeste': ['Minas Gerais', 'Espírito Santo', 'Rio de Janeiro', 'São Paulo'],
    'Sul': ['Paraná', 'Santa Catarina', 'Rio Grande do Sul'],
    'Centro-Oeste': ['Mato Grosso do Sul', 'Mato Grosso', 'Goiás', 'Distrito Federal']
}
regioes['Brasil'] = regioes['Norte'] + regioes['Nordeste'] + regioes['Sudeste'] + regioes['Sul'] + regioes['Centro-Oeste']

vals = {}
for ano in [1991, 2000, 2010]:
    for regiao in regioes:
        dados = Atlas[Atlas['Local'].isin(regioes[regiao]) & (Atlas['Ano'] == ano)]
        vals[(regiao, ano)] = sum(dados['Morte5'] * dados['Populacao']) / sum(dados['Populacao'])

local, ano, morte, nivel = [], [], [], []
for i, j in vals:
    local.append(i)
    ano.append(j)
    morte.append(vals[(i, j)])
    nivel.append('BR' if i == 'Brasil' else 'GR')

Atlas = pd.concat([Atlas.drop(columns='Populacao'),
                   pd.DataFrame({'Local': local, 'Ano': ano, 'Morte5': morte, 'Nível': nivel})])
Atlas = Atlas.sort_values(by=['Local', 'Ano'])
Atlas = Atlas.reset_index(drop=True)

In [39]:
Atlas.loc[(Atlas['Ano'] == 2010) & (Atlas['Morte5'] >= 25)]

,Local,Ano,Morte5,Nível
67,Alagoas,2010,31.03,UF
61,Maranhão,2010,30.61,UF


In [40]:
Atlas.loc[(Atlas['Ano'] == 1991) & (Atlas['Morte5'] < 25)]

,Local,Ano,Morte5,Nível


In [41]:
Atlas.loc[(Atlas['Ano'] == 2000) & (Atlas['Morte5'] < 25)]

,Local,Ano,Morte5,Nível
53,Distrito Federal,2000,24.040000,UF
47,Paraná,2000,23.530000,UF
49,Rio Grande do Sul,2000,19.440000,UF
45,Rio de Janeiro,2000,23.070000,UF
48,Santa Catarina,2000,19.370000,UF
8,Sudeste,2000,24.761942,GR
9,Sul,2000,20.981529,GR
46,São Paulo,2000,22.490000,UF


In [57]:
Atlas.to_csv('Dados Tratados/Atlas.csv', index=False)

# ODS Brasil - Sidra IBGE (Tabela 6695)

In [3]:
ods = pd.read_excel('tabela6695.xlsx')[['Brasil, Grande Região e Unidade da Federação', 'Ano', 'Unnamed: 4', 'Nível']]
ods.columns = ['Local', 'Ano', 'Morte5', 'Nível']
ods = ods.ffill(axis=0)
ods

,Local,Ano,Morte5,Nível
0,Brasil,1990,53.7,BR
1,Brasil,1991,50.7,BR
2,Brasil,1992,47.8,BR
3,Brasil,1993,45.2,BR
4,Brasil,1994,42.6,BR
...,...,...,...,...
1084,Distrito Federal,2018,11.5,UF
1085,Distrito Federal,2019,10.3,UF
1086,Distrito Federal,2020,10.7,UF
1087,Distrito Federal,2021,12.0,UF


In [58]:
ods.to_csv('Dados Tratados/ODS.csv', index=False)

# Atlas + ODS 2020

In [53]:
Atlas.loc[(Atlas['Ano'] == 2010)].head(5)

,Local,Ano,Morte5,Nível
55,Acre,2010,24.93,UF
67,Alagoas,2010,31.03,UF
59,Amapá,2010,16.16,UF
56,Amazonas,2010,18.22,UF
69,Bahia,2010,23.49,UF


In [52]:
ods.loc[(ods['Ano'] == 2020)].head(5)

,Local,Ano,Morte5,Nível
30,Brasil,2020,14.0,BR
63,Norte,2020,18.9,GR
96,Nordeste,2020,16.3,GR
129,Sudeste,2020,12.2,GR
162,Sul,2020,10.4,GR


In [55]:
Atlas_ods = pd.concat([Atlas, ods.loc[ods['Ano'] == 2020]])
Atlas_ods = Atlas_ods.sort_values(by=['Local', 'Ano'])
Atlas_ods = Atlas_ods.reset_index(drop=True)
Atlas_ods 

,Local,Ano,Morte5,Nível
0,Acre,1991,50.63,UF
1,Acre,2000,34.08,UF
2,Acre,2010,24.93,UF
3,Acre,2020,19.30,UF
4,Alagoas,1991,94.68,UF
...,...,...,...,...
127,São Paulo,2020,11.10,UF
128,Tocantins,1991,83.26,UF
129,Tocantins,2000,46.98,UF
130,Tocantins,2010,21.05,UF


In [59]:
Atlas_ods.to_csv('Dados Tratados/Atlas_ODS.csv', index=False)

# Atlas - Municípios de SP

In [75]:
Atlas_municipal = pd.read_csv('Atlas Municipios.csv', usecols=['Município', 'ANO', 'MORT5', 'UF'])
Atlas_municipal = Atlas_municipal.loc[Atlas_municipal['UF'] == 35] # SP
Atlas_municipal = Atlas_municipal[['Município', 'ANO', 'MORT5']]
Atlas_municipal.columns = ['Local', 'Ano', 'Morte5']
Atlas_municipal = Atlas_municipal.sort_values(by=['Local', 'Ano']).reset_index(drop=True)
Atlas_municipal

,Local,Ano,Morte5
0,ADAMANTINA,1991,"29,31"
1,ADAMANTINA,2000,"19,51"
2,ADAMANTINA,2010,"13,92"
3,ADOLFO,1991,"25,87"
4,ADOLFO,2000,"20,58"
...,...,...,...
1930,ÁLVARO DE CARVALHO,2000,"31,47"
1931,ÁLVARO DE CARVALHO,2010,"20,23"
1932,ÓLEO,1991,"30,55"
1933,ÓLEO,2000,"23,85"


In [77]:
Atlas_municipal.to_csv('Dados Tratados/Atlas_Municipios_SP.csv', index=False)

# Atlas - Vale do Paraíba e Litoral Norte

In [81]:
Atlas_vale = pd.read_excel('Atlas_Vale.xlsx', sheet_name='Sheet1', usecols=['UDH_Atlas', 'NOME_UDH', 'ANO', 'NOME_MUN', 'MORT5'])
Atlas_vale.columns = ['Cod_UDH', 'Local', 'Municipio', 'Ano', 'Morte5']
Atlas_vale = Atlas_vale.sort_values(by=['Cod_UDH', 'Ano']).reset_index(drop=True)
Atlas_vale

,Cod_UDH,Local,Municipio,Ano,Morte5
0,1350250700001,Jardim Paraíba / Jardim Santo Afonso,Aparecida (SP),2000,14.25
1,1350250700001,Jardim Paraíba / Jardim Santo Afonso,Aparecida (SP),2010,10.26
2,1350250700002,Rosa de Ouro,Aparecida (SP),2000,14.71
3,1350250700002,Rosa de Ouro,Aparecida (SP),2010,9.37
4,1350250700003,Vila Mariana : Centro Esportivo / Residencial ...,Aparecida (SP),2000,22.11
...,...,...,...,...,...
729,1355540600019,Estufa I / Estufa II,Ubatuba (SP),2010,18.83
730,1355540600020,Praia Vermelha / Praia Dura / Lázaro / Saco da...,Ubatuba (SP),2000,18.97
731,1355540600020,Praia Vermelha / Praia Dura / Lázaro / Saco da...,Ubatuba (SP),2010,13.94
732,1355540600021,Parque Estadual da Serra do Mar / Praia da For...,Ubatuba (SP),2000,18.97


In [82]:
Atlas_vale.to_csv('Dados Tratados/Atlas_Vale.csv', index=False)

# ODS Brasil - Ano que Bateu a Meta

In [11]:
ano = ods.copy()
ano = ano.loc[ano['Morte5'] <= 25]
ano['Anterior'] = ano['Local'].shift(1)
ano = ano.loc[ano['Local'] != ano['Anterior']]
ano = ano[['Local', 'Ano', 'Morte5', 'Nível']]
ano

,Local,Ano,Morte5,Nível
14,Brasil,2004,25.0,BR
53,Norte,2010,25.0,GR
85,Nordeste,2009,23.5,GR
108,Sudeste,1999,24.2,GR
138,Sul,1996,25.0,GR
176,Centro-Oeste,2001,24.9,GR
216,Rondônia,2008,24.8,UF
251,Acre,2010,24.0,UF
285,Amazonas,2011,24.0,UF
309,Roraima,2002,24.5,UF


In [12]:
ano.to_csv('../Dados Tratados/Ano_Conclusao.csv', index=False)